## 1. Încărcarea dataset-urilor finale


În această etapă sunt utilizate seturile de date rezultate din pașii anteriori ai proiectului (indicatori WDI și EIA), exportate în format CSV. Aceste tabele reprezintă datele brute care vor fi supuse procesului de curățare și agregare.

Înainte de procesarea propriu-zisă, sunt verificate structura și consistența inițială a seturilor de date pentru a confirma prezența variabilelor-cheie (țară, an, indicatori) și compatibilitatea între surse.


In [24]:
required_wdi_cols = {"country", "year"}
required_eia_cols = {"countryRegionName", "period", "productId"}

print("WDI missing cols:", required_wdi_cols - set(wdi.columns))
print("EIA missing cols:", required_eia_cols - set(eia.columns))

WDI missing cols: set()
EIA missing cols: {'countryRegionName', 'period'}


In [43]:
print("Ani WDI:", sorted(wdi["year"].unique()))

# EIA: în dataset-ul tău coloana cu anul este "year"
print("Ani EIA:", sorted(pd.to_numeric(eia["year"], errors="coerce").dropna().astype(int).unique()))


Ani WDI: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]
Ani EIA: [np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023)]


In [45]:
import pandas as pd

wdi = pd.read_csv("wdi_indicators_wide_normalized.csv")
eia = pd.read_csv("eia_raw_project.csv")

wdi.head(), eia.head()

(    country  year  electricity_access  energy_use_pc  gdp_per_capita  \
 0  Bulgaria  2020           -3.871615      -0.218166       -1.667680   
 1  Bulgaria  2021           -2.484768       0.168058       -1.244180   
 2  Bulgaria  2022            0.288926       0.243583       -1.046599   
 3  Bulgaria  2023            0.288926      -0.309816       -0.684267   
 4   Czechia  2020            0.288926       1.451936        0.775115   
 
    renewables_share  
 0          0.854690  
 1          0.636471  
 2               NaN  
 3               NaN  
 4         -0.423448  ,
    period  productId     productName  activityId activityName countryRegionId  \
 0    2023         44  Primary energy           1   Production             BGR   
 1    2023         44  Primary energy           1   Production             BGR   
 2    2023         44  Primary energy           1   Production             BGR   
 3    2023         44  Primary energy           1   Production             CZE   
 4    2023 

## 2. Armonizarea cheilor (țară – an)


În această etapă a fost realizată armonizarea cheilor de legătură între seturile de date WDI și EIA, pentru a permite îmbinarea ulterioară a acestora într-un dataset comun.

Am ales ca unitate de analiză combinația țară × an, comună ambelor surse de date.

A fost aleasă ca unitate de analiză combinația țară × an, comună ambelor surse de date. Denumirile diferite ale variabilelor-cheie au fost uniformizate (countryRegionName → country, period → year), iar coloana anului a fost convertită la tip numeric (int) pentru a asigura consistența operațiilor ulterioare.

S-a verificat că perechile (țară, an) sunt unice în ambele seturi de date, prevenind astfel dublarea observațiilor în timpul agregării și îmbinării dataseturilor.

In [116]:
wdi[["country", "year"]].drop_duplicates().shape
eia[["country", "year"]].drop_duplicates().shape


(24, 2)

In [117]:
eia = eia.rename(columns={
    "countryRegionName": "country",
    "period": "year"
})

eia["year"] = eia["year"].astype(int)

## 3. Agregarea indicatorilor EIA


Indicatorii EIA sunt raportați în unități diferite (ex. terajouli, milioane tone echivalent petrol). Pentru a permite agregarea și comparația între țări și ani, valorile au fost standardizate folosind Z-score înainte de agregare.

In [118]:
eia = eia.copy()
eia["indicator"] = eia["productName"]  # sau eia["productId"]

Agregarea a fost realizată folosind media valorilor normalizate, deoarece datele sunt deja anuale și nu reprezintă fluxuri care trebuie însumate în timp.

In [119]:
eia = eia.copy()
eia["value"] = pd.to_numeric(eia["value"], errors="coerce")

eia["value_norm"] = (
    eia.groupby(["productName", "unitName"])["value"]
       .transform(lambda x: (x - x.mean()) / x.std(ddof=0))
)

In [120]:
eia_agg = (
    eia
    .groupby(["countryRegionId", "year", "productName"], as_index=False)
    .agg(value_norm=("value_norm", "mean"))
)

eia_agg.head()


,countryRegionId,year,productName,value_norm
0,BGR,2020,CO2 emissions,-0.645749
1,BGR,2020,Energy intensity,0.511984
2,BGR,2020,Primary energy,-0.641981
3,BGR,2021,CO2 emissions,-0.583621
4,BGR,2021,Energy intensity,0.762912


In [121]:
eia_agg = eia_agg.rename(columns={"countryRegionId": "country"})


## 4. Integrarea WDI și EIA


În această etapă, seturile de date WDI și EIA au fost integrate într-un
dataset final comun, utilizând cheia (țară, an).

Îmbinarea a fost realizată printr-un left join, folosind datele WDI ca bază,
pentru a păstra structura completă a indicatorilor socio-economici
și a adăuga informațiile energetice disponibile din EIA.

In [122]:
df_final = pd.merge(
    wdi,
    eia_agg,
    on=["country", "year"],
    how="left"
)

df_final.head()

,country,year,electricity_access,energy_use_pc,gdp_per_capita,renewables_share,productName,value_norm
0,Bulgaria,2020,-3.871615,-0.218166,-1.667680,0.854690,NaN,NaN
1,Bulgaria,2021,-2.484768,0.168058,-1.244180,0.636471,NaN,NaN
2,Bulgaria,2022,0.288926,0.243583,-1.046599,NaN,NaN,NaN
3,Bulgaria,2023,0.288926,-0.309816,-0.684267,NaN,NaN,NaN
4,Czechia,2020,0.288926,1.451936,0.775115,-0.423448,NaN,NaN


Valorile lipsă rezultate după integrare indică ani sau țări pentru care
nu există date EIA disponibile și nu reprezintă erori de procesare.

## 5. Verificarea consistenței datelor


Dataset-ul agregat final a fost salvat pentru a fi utilizat în etapele
ulterioare ale proiectului (analiză, vizualizare sau încărcare în baza de date),
sub denumirea energy_panel_final.

In [123]:
df_final.isna().sum()
df_final.describe()

,year,electricity_access,energy_use_pc,gdp_per_capita,renewables_share,value_norm
count,24.00000,2.400000e+01,2.400000e+01,2.400000e+01,1.200000e+01,0.0
mean,2021.50000,-6.566969e-14,-1.110223e-16,2.590520e-16,-1.480297e-16,NaN
std,1.14208,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN
min,2020.00000,-3.871615e+00,-1.833422e+00,-1.667680e+00,-1.109278e+00,NaN
25%,2020.75000,2.889265e-01,-2.699439e-01,-6.999383e-01,-7.663633e-01,NaN
50%,2021.50000,2.889265e-01,2.448017e-02,-1.305348e-01,-2.987518e-01,NaN
75%,2022.25000,2.889265e-01,3.873331e-01,5.278344e-01,6.910259e-01,NaN
max,2023.00000,2.889265e-01,1.895070e+00,2.336249e+00,1.789913e+00,NaN


In [124]:
eia_agg["value_norm"].describe()

count    7.200000e+01
mean    -1.110223e-16
std      9.467501e-01
min     -1.965317e+00
25%     -5.850194e-01
50%     -2.123678e-01
75%      5.165611e-01
max      2.329097e+00
Name: value_norm, dtype: float64

In [125]:
eia_agg.columns


Index(['country', 'year', 'productName', 'value_norm'], dtype='object')

In [126]:
iso3_to_country = {
    "BGR": "Bulgaria",
    "CZE": "Czechia",
    "HUN": "Hungary",
    "POL": "Poland",
    "ROU": "Romania",
    "SVK": "Slovakia"
}

eia_agg["country"] = eia_agg["country"].map(iso3_to_country)

In [127]:
print(sorted(eia_agg["country"].unique()))
print(sorted(wdi["country"].unique()))

['Bulgaria', 'Czechia', 'Hungary', 'Poland', 'Romania', 'Slovakia']
['Bulgaria', 'Czechia', 'Hungary', 'Poland', 'Romania', 'Slovakia']


In [128]:
df_final = (
    wdi
    .merge(
        eia_agg,
        on=["country", "year"],
        how="left"
    )
)

In [129]:
df_final["value_norm"].isna().sum()
df_final["value_norm"].describe()

count    7.200000e+01
mean    -1.110223e-16
std      9.467501e-01
min     -1.965317e+00
25%     -5.850194e-01
50%     -2.123678e-01
75%      5.165611e-01
max      2.329097e+00
Name: value_norm, dtype: float64

In [132]:
df_final.isna().sum()
df_final.describe()

,year,electricity_access,energy_use_pc,gdp_per_capita,renewables_share,value_norm
count,72.00000,7.200000e+01,7.200000e+01,7.200000e+01,3.600000e+01,7.200000e+01
mean,2021.50000,-6.563885e-14,-1.480297e-16,2.837237e-16,-1.480297e-16,-1.110223e-16
std,1.12588,9.858149e-01,9.858149e-01,9.858149e-01,9.710083e-01,9.467501e-01
min,2020.00000,-3.871615e+00,-1.833422e+00,-1.667680e+00,-1.109278e+00,-1.965317e+00
25%,2020.75000,2.889265e-01,-2.699439e-01,-6.999383e-01,-7.663633e-01,-5.850194e-01
50%,2021.50000,2.889265e-01,2.448017e-02,-1.305348e-01,-2.987518e-01,-2.123678e-01
75%,2022.25000,2.889265e-01,3.873331e-01,5.278344e-01,6.910259e-01,5.165611e-01
max,2023.00000,2.889265e-01,1.895070e+00,2.336249e+00,1.789913e+00,2.329097e+00
